In [1]:
from pyspark.sql import SQLContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

**Read in 'buy-clicks.csv' and 'combined_data.csv' files...**

In [2]:
spark = SQLContext(sc)

buy_clicks = sqlContext.read.load('file:///home/cloudera/Downloads/big_data_capstone_datasets_and_scripts/flamingo-data/buy-clicks.csv', 
                        format='com.databricks.spark.csv', 
                        header='true',inferSchema='true')

combined_data = sqlContext.read.load('file:///home/cloudera/Downloads/big_data_capstone_datasets_and_scripts/combined-data.csv', 
                        format='com.databricks.spark.csv', 
                        header='true',inferSchema='true')

In [5]:
buy_clicks.columns

['timestamp', 'txId', 'userSessionId', 'team', 'userId', 'buyId', 'price']

In [6]:
buy_clicks.count()

2947

In [32]:
type(buy_clicks)

pyspark.sql.dataframe.DataFrame

**Aggregate total purchases by userId by summing up the price column in 'buy_clicks'**

In [35]:
total_purchases = buy_clicks.groupBy('userId').sum('price').show()
type(total_purchases)

+------+----------+
|userId|sum(price)|
+------+----------+
|   231|      63.0|
|  2032|      20.0|
|   233|      28.0|
|  1234|      53.0|
|  1634|      27.0|
|    34|      95.0|
|  1434|       9.0|
|  1835|      27.0|
|  2035|      40.0|
|  1235|      40.0|
|  1436|      16.0|
|   436|      43.0|
|  1636|      25.0|
|   236|      43.0|
|  2236|      15.0|
|  1837|      67.0|
|    38|      30.0|
|  1639|     155.0|
|   239|      20.0|
|   439|      25.0|
+------+----------+
only showing top 20 rows



NoneType

**Convert 'total_purchases' to a dataframe...**

In [36]:
total_purchases = buy_clicks.groupBy('userId').sum('price')
type(total_purchases)

pyspark.sql.dataframe.DataFrame

In [17]:
combined_data.columns

['userId',
 'userSessionId',
 'teamLevel',
 'platformType',
 'count_gameclicks',
 'count_hits',
 'count_buyId',
 'avg_price']

In [18]:
combined_data.count()

4619

In [28]:
type(combined_data)

pyspark.sql.dataframe.DataFrame

**Aggregate counts of game_clicks and flamingo hits by userId by summing up the appropriate columns in 'combined_data'**

In [44]:
combined_data_by_userid = combined_data.groupby('userId').sum('count_gameclicks','count_hits').show()

+------+---------------------+---------------+
|userId|sum(count_gameclicks)|sum(count_hits)|
+------+---------------------+---------------+
|  1631|                  392|             45|
|  2231|                  395|             38|
|  2031|                 2271|            227|
|   231|                  262|             28|
|  2032|                  638|             59|
|   832|                 1469|            141|
|    32|                  281|             25|
|  1632|                  309|             30|
|   432|                   32|              3|
|  1433|                  372|             53|
|   233|                  250|             29|
|  2033|                  575|             80|
|   433|                   34|              2|
|  1034|                  376|             45|
|  1834|                  510|             47|
|  1234|                  590|             73|
|    34|                  665|             79|
|  1634|                 2546|            266|
|  1434|     

**Convert 'combined_data_by_userid' to a dataframe...**

In [45]:
combined_data_by_userid = combined_data.groupby('userId').sum('count_gameclicks','count_hits')

**Give alias names to the two datasets to be joined...**

In [56]:
inner_join = combined_data_by_userid.alias("a").join(
    total_purchases.alias("b"), combined_data_by_userid['userId'] == 
    total_purchases['userId']).select("a.userId", "a.sum(count_gameclicks)",
                                      "a.sum(count_hits)", "b.sum(price)").show()

+------+---------------------+---------------+----------+
|userId|sum(count_gameclicks)|sum(count_hits)|sum(price)|
+------+---------------------+---------------+----------+
|   231|                  262|             28|      63.0|
|  2032|                  638|             59|      20.0|
|   233|                  250|             29|      28.0|
|    34|                  665|             79|      95.0|
|  1234|                  590|             73|      53.0|
|  1434|                  772|             89|       9.0|
|  1634|                 2546|            266|      27.0|
|  1235|                  367|             39|      40.0|
|  1835|                  734|             94|      27.0|
|   236|                  606|             70|      43.0|
|   436|                 4392|            494|      43.0|
|  1436|                  622|             65|      16.0|
|  1636|                  317|             29|      25.0|
|  2236|                  299|             36|      15.0|
|  1837|      

**So, we will develop clusters based on total gameclicks, total hits, and total purchases by userId.**